<a href="https://colab.research.google.com/github/RuslanMavlitov/IDE/blob/master/tsym_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Задача проанализировать суточное потребление по восьми трансформаторам по логам с счетчиков на низкой стороне трансформаторов.

In [5]:
# Загрузим библиотеки для работы с данными
import pandas as pd
import numpy as np
# Будем использовать Plotly для визуализации
import plotly.express as px
import plotly.graph_objects as go
# Все данные будем брать с Гугл-диска
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Анализ данных

### 1.1 Чтение данных

In [74]:
#Загрузим данные
T1 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T1_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T2 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T2_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T3 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T3_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T4 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T4_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T5 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T5_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T6 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T6_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T7 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T7_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')
T8 = pd.read_csv('/content/drive/MyDrive/TSYM/03-04.07.2024_T8/T8_03_04.07.2024.csv', encoding='iso-8859-1', delimiter=';')

In [ ]:
#Посмотрим что получили
T1.head()

,Äàòà,Âðåìÿ,Òî÷êà,Çíà÷åíèå,Òèï
0,03.07.2024,16:31:05.000,SHA_GRSCH_RP4_PK47_AI210,<634>,NaN
1,03.07.2024,16:31:05.000,SHA_GRSCH_RP4_PK47_AI212,<651>,NaN
2,03.07.2024,16:32:25.000,SHA_GRSCH_RP4_PK47_AI210,<647>,NaN
3,03.07.2024,16:32:33.000,SHA_GRSCH_RP4_PK47_AI210,<617>,NaN
4,03.07.2024,16:32:39.000,SHA_GRSCH_RP4_PK47_AI212,<638>,NaN


Данные нужно преобразовать:
- переименуем столбцы
- дату и время в один формат по времени
- токи в счетчике переименуем в номера фаз
- удалим лишние знаки в значении тока


### 1.2 Обработка данных

In [75]:
#Удалим последний столбец без данных
T1.drop(T1.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T1.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T1['Current'] = T1['Current'].str.replace('<', '', regex=True)
T1['Current'] = T1['Current'].str.replace('>', '', regex=True)
T1['Current'] = T1['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T1['datetime'] = pd.to_datetime(T1['data'] + " " + T1['Time'])
T1['datetime'] = T1['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T1 = T1.set_index(pd.DatetimeIndex(T1['datetime']))
T1 = T1.drop(['datetime'], axis = 1)
T1.drop(['data','Time'], axis=1, inplace=True)

# Добавим столбец "Мощность" умножив ток на 400В и примем cos(fi) = 1
T1['Power_kVA'] = T1['Current']*400*(3**0.5)/1000

# Добавим столбец со оставшейся мощность от номинальной загрузки 65% от Sном (650 kVA)
T1['Free_Power_kVA'] = 650 - T1['Power_kVA']

In [ ]:
#Посмотрим сколько уникальных имен датчиков тока в данных
print(f"В данных всего {T1['Multimetr_name'].nunique()} уникальных датчика тока")

В данных всего 2 уникальных датчика тока


In [76]:
T1['Multimetr_name'] = T1['Multimetr_name'].replace({'SHA_GRSCH_RP4_PK47_AI210' : 'Phase_1', 'SHA_GRSCH_RP4_PK47_AI212' : 'Phase_2'})

In [49]:
#Посмотрим на результат
T1.head(2)

,Multimetr_name,Current,Power_kVA,Free_Power_kVA
datetime,,,,
2024-07-03 16:31:05,Phase_1,634.0,439.248085,210.751915
2024-07-03 16:31:05,Phase_2,651.0,451.026030,198.973970


Уже намного лучше... Проделаем эту обработку по всем счетчикам

In [77]:
#Удалим последний столбец без данных
T2.drop(T2.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T2.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T2['Current'] = T2['Current'].str.replace('<', '', regex=True)
T2['Current'] = T2['Current'].str.replace('>', '', regex=True)
T2['Current'] = T2['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T2['datetime'] = pd.to_datetime(T2['data'] + " " + T2['Time'])
T2['datetime'] = T2['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T2 = T2.set_index(pd.DatetimeIndex(T2['datetime']))
T2 = T2.drop(['datetime'], axis = 1)
T2.drop(['data','Time'], axis=1, inplace=True)

T2['Multimetr_name'] = T2['Multimetr_name'].replace(
    {'SHA_GRSCH_RP3_PK47_AI223' : 'Phase_1',
     'SHA_GRSCH_RP3_PK47_AI224' : 'Phase_2',
     'SHA_GRSCH_RP3_PK47_AI225' : 'Phase_3'})

# Добавим столбец "Мощность" умножив ток на 400В и примем cos(fi) = 1
T2['Power_kVA'] = T2['Current']*400*(3**0.5)/1000

# Добавим столбец со оставшейся мощность от номинальной загрузки 65% от Sном (650 kVA)
T2['Free_Power_kVA'] = 650 - T2['Power_kVA']

In [78]:
#Удалим последний столбец без данных
T3.drop(T3.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T3.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T3['Current'] = T3['Current'].str.replace('<', '', regex=True)
T3['Current'] = T3['Current'].str.replace('>', '', regex=True)
T3['Current'] = T3['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T3['datetime'] = pd.to_datetime(T3['data'] + " " + T3['Time'])
T3['datetime'] = T3['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T3 = T3.set_index(pd.DatetimeIndex(T3['datetime']))
T3 = T3.drop(['datetime'], axis = 1)
T3.drop(['data','Time'], axis=1, inplace=True)

T3['Multimetr_name'] = T3['Multimetr_name'].replace(
    {'SHA_GRSCH_RP2_PK47_AI236' : 'Phase_1',
     'SHA_GRSCH_RP2_PK47_AI237' : 'Phase_2',
     'SHA_GRSCH_RP2_PK47_AI238' : 'Phase_3'})

# Добавим столбец "Мощность" умножив ток на 400В и примем cos(fi) = 1
T3['Power_kVA'] = T3['Current']*400*(3**0.5)/1000

# Добавим столбец со оставшейся мощность от номинальной загрузки 65% от Sном (650 kVA)
T3['Free_Power_kVA'] = 650 - T3['Power_kVA']

In [79]:
#Удалим последний столбец без данных
T4.drop(T4.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T4.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T4['Current'] = T4['Current'].str.replace('<', '', regex=True)
T4['Current'] = T4['Current'].str.replace('>', '', regex=True)
T4['Current'] = T4['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T4['datetime'] = pd.to_datetime(T4['data'] + " " + T4['Time'])
T4['datetime'] = T4['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T4 = T4.set_index(pd.DatetimeIndex(T4['datetime']))
T4 = T4.drop(['datetime'], axis = 1)
T4.drop(['data','Time'], axis=1, inplace=True)

T4['Multimetr_name'] = T4['Multimetr_name'].replace(
    {'SHA_GRSCH_RP1_PK47_AI249' : 'Phase_1',
     'SHA_GRSCH_RP1_PK47_AI250' : 'Phase_2',
     'SHA_GRSCH_RP1_PK47_AI251' : 'Phase_3'})

# Добавим столбец "Мощность" умножив ток на 400В и примем cos(fi) = 1
T4['Power_kVA'] = T4['Current']*400*(3**0.5)/1000

# Добавим столбец со оставшейся мощность от номинальной загрузки 65% от Sном (650 kVA)
T4['Free_Power_kVA'] = 650 - T4['Power_kVA']

In [80]:
#Удалим последний столбец без данных
T5.drop(T5.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T5.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T5['Current'] = T5['Current'].str.replace('<', '', regex=True)
T5['Current'] = T5['Current'].str.replace('>', '', regex=True)
T5['Current'] = T5['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T5['datetime'] = pd.to_datetime(T5['data'] + " " + T5['Time'])
T5['datetime'] = T5['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T5 = T5.set_index(pd.DatetimeIndex(T5['datetime']))
T5 = T5.drop(['datetime'], axis = 1)
T5.drop(['data','Time'], axis=1, inplace=True)

T5['Multimetr_name'] = T5['Multimetr_name'].replace(
    {'GRSCH_PM31_A1' : 'Phase_1',
     'GRSCH_PM31_A2' : 'Phase_2',
     'GRSCH_PM31_A3' : 'Phase_3'})

# Добавим столбец "Мощность" умножив ток на 400В и примем cos(fi) = 1
T5['Power_kVA'] = T5['Current']*400*(3**0.5)/1000

# Добавим столбец со оставшейся мощность от номинальной загрузки 65% от Sном (1040 kVA)
T5['Free_Power_kVA'] = 1040 - T5['Power_kVA']

In [81]:
#Удалим последний столбец без данных
T6.drop(T6.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T6.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T6['Current'] = T6['Current'].str.replace('<', '', regex=True)
T6['Current'] = T6['Current'].str.replace('>', '', regex=True)
T6['Current'] = T6['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T6['datetime'] = pd.to_datetime(T6['data'] + " " + T6['Time'])
T6['datetime'] = T6['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T6 = T6.set_index(pd.DatetimeIndex(T6['datetime']))
T6 = T6.drop(['datetime'], axis = 1)
T6.drop(['data','Time'], axis=1, inplace=True)

T6['Multimetr_name'] = T6['Multimetr_name'].replace(
    {'GRSCH_PM32_A1' : 'Phase_1',
     'GRSCH_PM32_A2' : 'Phase_2',
     'GRSCH_PM32_A3' : 'Phase_3'})

# Добавим столбец "Мощность" умножив ток на 400В и примем cos(fi) = 1
T6['Power_kVA'] = T6['Current']*400*(3**0.5)/1000

# Добавим столбец со оставшейся мощность от номинальной загрузки 65% от Sном (1040 kVA)
T6['Free_Power_kVA'] = 1040 - T6['Power_kVA']

In [82]:
#Удалим последний столбец без данных
T7.drop(T7.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T7.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T7['Current'] = T7['Current'].str.replace('<', '', regex=True)
T7['Current'] = T7['Current'].str.replace('>', '', regex=True)
T7['Current'] = T7['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T7['datetime'] = pd.to_datetime(T7['data'] + " " + T7['Time'])
T7['datetime'] = T7['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T7 = T7.set_index(pd.DatetimeIndex(T7['datetime']))
T7 = T7.drop(['datetime'], axis = 1)
T7.drop(['data','Time'], axis=1, inplace=True)

T7['Multimetr_name'] = T7['Multimetr_name'].replace(
    {'GRSCH_PM41_A1' : 'Phase_1',
     'GRSCH_PM41_A2' : 'Phase_2',
     'GRSCH_PM41_A3' : 'Phase_3'})

# Добавим столбец "Мощность" умножив ток на 400В и примем cos(fi) = 1
T7['Power_kVA'] = T7['Current']*400*(3**0.5)/1000

# Добавим столбец со оставшейся мощность от номинальной загрузки 65% от Sном (1040 kVA)
T7['Free_Power_kVA'] = 1040 - T7['Power_kVA']

In [83]:
#Удалим последний столбец без данных
T8.drop(T8.columns[4], axis=1, inplace=True)

#Переименуем столбцы
T8.columns =['data', 'Time', 'Multimetr_name', 'Current' ]

#Подредактируем значения по току
T8['Current'] = T8['Current'].str.replace('<', '', regex=True)
T8['Current'] = T8['Current'].str.replace('>', '', regex=True)
T8['Current'] = T8['Current'].astype(float).apply(np.floor)

# Создадим общую колонку с временем
T8['datetime'] = pd.to_datetime(T8['data'] + " " + T8['Time'])
T8['datetime'] = T8['datetime'].dt.strftime('%d/%m/%Y %H:%M:%S')
T8 = T8.set_index(pd.DatetimeIndex(T8['datetime']))
T8 = T8.drop(['datetime'], axis = 1)
T8.drop(['data','Time'], axis=1, inplace=True)

T8['Multimetr_name'] = T8['Multimetr_name'].replace(
    {'GRSCH_PM42_A1' : 'Phase_1',
     'GRSCH_PM42_A2' : 'Phase_2',
     'GRSCH_PM42_A3' : 'Phase_3'})

# Добавим столбец "Мощность" умножив ток на 400В и примем cos(fi) = 1
T8['Power_kVA'] = T8['Current']*400*(3**0.5)/1000

# Добавим столбец со оставшейся мощность от номинальной загрузки 65% от Sном (1040 kVA)
T8['Free_Power_kVA'] = 1040 - T8['Power_kVA']

### 1.3 Визуализация данных

In [93]:
# Построим график нагрузки Т-1
fig = px.line(T1, x=T1.index, y="Power_kVA", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-1",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Мощность, А"),
    )
fig.add_hline(y=1000, line_dash="dot",
              annotation_text="100% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1000/2,  line_dash="dot",
              annotation_text="50% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1000*0.65,  line_dash="dot",
              annotation_text="65% от Sном",
              annotation_position="bottom right")

fig.show()

In [94]:
# Построим график нагрузки Т-2
fig = px.line(T2, x=T2.index, y="Power_kVA", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-2",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Мощность, А"),
    )
fig.add_hline(y=1000, line_dash="dot",
              annotation_text="100% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1000/2,  line_dash="dot",
              annotation_text="50% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1000*0.65,  line_dash="dot",
              annotation_text="65% от Sном",
              annotation_position="bottom right")

fig.show()

In [95]:
# Построим график нагрузки Т-3
fig = px.line(T3, x=T3.index, y="Power_kVA", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-3",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Мощность, А"),
    )
fig.add_hline(y=1000, line_dash="dot",
              annotation_text="100% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1000/2,  line_dash="dot",
              annotation_text="50% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1000*0.65,  line_dash="dot",
              annotation_text="65% от Sном",
              annotation_position="bottom right")

fig.show()

In [96]:
# Построим график нагрузки Т-4
fig = px.line(T4, x=T4.index, y="Power_kVA", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-4",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Мощность, А"),
    )
fig.add_hline(y=1000, line_dash="dot",
              annotation_text="100% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1000/2,  line_dash="dot",
              annotation_text="50% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1000*0.65,  line_dash="dot",
              annotation_text="65% от Sном",
              annotation_position="bottom right")

fig.show()

In [98]:
# Построим график нагрузки Т-5
fig = px.line(T5, x=T5.index, y="Power_kVA", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-5",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Мощность, А"),
    )
fig.add_hline(y=1600, line_dash="dot",
              annotation_text="100% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1600/2,  line_dash="dot",
              annotation_text="50% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1600*0.65,  line_dash="dot",
              annotation_text="65% от Sном",
              annotation_position="bottom right")

fig.show()

In [99]:
# Построим график нагрузки Т-6
fig = px.line(T6, x=T6.index, y="Power_kVA", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-6",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Мощность, А"),
    )
fig.add_hline(y=1600, line_dash="dot",
              annotation_text="100% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1600/2,  line_dash="dot",
              annotation_text="50% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1600*0.65,  line_dash="dot",
              annotation_text="65% от Sном",
              annotation_position="bottom right")

fig.show()

In [100]:
# Построим график нагрузки Т-7
fig = px.line(T7, x=T7.index, y="Power_kVA", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-7",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Мощность, А"),
    )
fig.add_hline(y=1600, line_dash="dot",
              annotation_text="100% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1600/2,  line_dash="dot",
              annotation_text="50% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1600*0.65,  line_dash="dot",
              annotation_text="65% от Sном",
              annotation_position="bottom right")

fig.show()

In [101]:
# Построим график нагрузки Т-8
fig = px.line(T8, x=T8.index, y="Power_kVA", color='Multimetr_name')
fig.update_layout(
    title={
        'text': "Загрузка Т-8",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis=dict(title="Время"),
    yaxis=dict(title="Мощность, А"),
    )
fig.add_hline(y=1600, line_dash="dot",
              annotation_text="100% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1600/2,  line_dash="dot",
              annotation_text="50% от Sном",
              annotation_position="bottom right")
fig.add_hline(y=1600*0.65,  line_dash="dot",
              annotation_text="65% от Sном",
              annotation_position="bottom right")

fig.show()

### 1.4 Получение результатов

In [45]:
T1.describe()

,Current,Power kVA,Free_Power_kVA
count,2178.000000,2178.000000,2178.000000
mean,573.196051,397.121874,252.878126
std,61.497502,42.606719,42.606719
min,463.000000,320.775810,146.319625
25%,505.000000,349.874263,216.294478
50%,591.000000,409.456811,240.543189
75%,626.000000,433.705522,300.125737
max,727.000000,503.680375,329.224190


**Итог по Т-1**

За суточный минимум (ночное время) дополнительная мощность составляет 329 кВА.

В часы максимумов 146 кВА

В среднем 252 кВА.


In [66]:
T2.describe()

,Current,Power_kVA,Free_Power_kVA
count,3347.000000,3347.000000,3347.000000
mean,587.867045,407.286236,242.713764
std,101.369068,70.230550,70.230550
min,383.000000,265.350184,26.461709
25%,510.000000,353.338365,193.431407
50%,591.000000,409.456811,240.543189
75%,659.000000,456.568593,296.661635
max,900.000000,623.538291,384.649816


**Итог по Т-2**

За суточный минимум (ночное время) дополнительная мощность составляет 384 кВА.

В часы максимумов 26 кВА

В среднем 242 кВА.

In [67]:
T3.describe()

,Current,Power_kVA,Free_Power_kVA
count,3161.000000,3161.000000,3161.000000
mean,588.540652,407.752924,242.247076
std,99.613197,69.014047,69.014047
min,400.000000,277.128129,41.703756
25%,508.000000,351.952724,181.653462
50%,588.000000,407.378350,242.621650
75%,676.000000,468.346538,298.047276
max,878.000000,608.296244,372.871871


**Итог по Т-3**

За суточный минимум (ночное время) дополнительная мощность составляет 372 кВА.

В часы максимумов 41 кВА

В среднем 242 кВА.

In [68]:
T4.describe()

,Current,Power_kVA,Free_Power_kVA
count,3544.000000,3544.000000,3544.000000
mean,694.405474,481.098225,168.901775
std,136.612623,94.648002,94.648002
min,414.000000,286.827614,2.905818
25%,580.000000,401.835787,95.743742
50%,738.000000,511.301398,138.698602
75%,800.000000,554.256258,248.164213
max,934.000000,647.094182,363.172386


**Итог по Т-4**

За суточный минимум (ночное время) дополнительная мощность составляет 363 кВА.

В часы максимумов 2 кВА

В среднем 168 кВА.

In [88]:
T5.describe()

,Current,Power_kVA,Free_Power_kVA
count,16377.000000,16377.000000,16377.000000
mean,517.545704,358.566182,681.433818
std,117.979729,81.738754,81.738754
min,319.000000,221.009683,482.972460
25%,381.000000,263.964543,617.379603
50%,557.000000,385.900920,654.099080
75%,610.000000,422.620397,776.035457
max,804.000000,557.027540,818.990317


**Итог по Т-5**

За суточный минимум (ночное время) дополнительная мощность составляет 818 кВА.

В часы максимумов 482 кВА

В среднем 681 кВА.

In [89]:
T6.describe()

,Current,Power_kVA,Free_Power_kVA
count,16377.000000,16377.000000,16377.000000
mean,472.116627,327.091994,712.908006
std,114.520991,79.342470,79.342470
min,285.000000,197.453792,528.698602
25%,347.000000,240.408652,640.935494
50%,502.000000,347.795802,692.204198
75%,576.000000,399.064506,799.591348
max,738.000000,511.301398,842.546208


**Итог по Т-6**

За суточный минимум (ночное время) дополнительная мощность составляет 842 кВА.

В часы максимумов 528 кВА

В среднем 712 кВА.

In [90]:
T7.describe()

,Current,Power_kVA,Free_Power_kVA
count,16377.000000,16377.000000,16377.000000
mean,926.109666,641.627598,398.372402
std,163.504010,113.278901,113.278901
min,623.000000,431.627061,196.144847
25%,752.000000,521.000883,290.368410
50%,969.000000,671.342893,368.657107
75%,1082.000000,749.631590,518.999117
max,1218.000000,843.855153,608.372939


**Итог по Т-7**

За суточный минимум (ночное время) дополнительная мощность составляет 608 кВА.

В часы максимумов 196 кВА

В среднем 398 кВА.

In [92]:
T8.describe()

,Current,Power_kVA,Free_Power_kVA
count,16371.000000,16371.000000,16371.000000
mean,719.775273,498.674937,541.325063
std,144.924371,100.406550,100.406550
min,374.000000,259.114801,342.329935
25%,571.000000,395.600404,453.874007
50%,778.000000,539.014211,500.985789
75%,846.000000,586.125993,644.399596
max,1007.000000,697.670065,780.885199


**Итог по Т-8**

За суточный минимум (ночное время) дополнительная мощность составляет 390 кВА.

В часы максимумов Т-8 загружен на 70% кВА, доп. мощности нет

В среднем 151 кВА.